# Exercises

Sourcing the Zillow database as the data source, create a jupyter notebook named explore_zillow and do the following:

- Ask at least 5 questions about the data, keeping in mind that your target variable is logerror. e.g. Is logerror significantly different for properties in LA County vs Orange County vs Ventura County?

- Answer those questions through a mix of statistical tests and visualizations.

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

### Imports

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Statistical Tests
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

pd.options.display.float_format = '{:20,.2f}'.format

import env
import explore
import acquire
import summarize
from summarize import df_summary

In [3]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

### Acquire/Wrangle Data

In [4]:
df = acquire.get_zillow_data()

df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1087254,10711855,nan,nan,nan,2.00,3.00,None,8.00,2.00,...,"60,371,132,321,007.00",-0.01,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,1072280,10711877,1.00,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,007.00",0.02,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,1340933,10711888,1.00,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,007.00",0.08,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,1878109,10711910,nan,nan,nan,2.00,3.00,None,8.00,2.00,...,"60,371,132,321,008.00",-0.04,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,2190858,10711923,nan,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,008.00",-0.01,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [5]:
info, describe, nulls, value_count = df_summary (df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72236 entries, 0 to 72235
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            72236 non-null  int64  
 1   parcelid                      72236 non-null  int64  
 2   airconditioningtypeid         23078 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   72236 non-null  float64
 6   bedroomcnt                    72236 non-null  float64
 7   buildingclasstypeid           0 non-null      object 
 8   buildingqualitytypeid         45079 non-null  float64
 9   calculatedbathnbr             71983 non-null  float64
 10  decktypeid                    612 non-null    float64
 11  finishedfloor1squarefeet      6030 non-null   float64
 12  calculatedfinishedsquarefeet  72051 non-null  float64
 13  f

In [6]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [7]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

In [8]:
# Function to read and wrangle data:

def wrangle_zillow():
    df = acquire.get_zillow_data()
    
    # Restrict df to only properties that meet single unit use criteria
    single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
    df = df[df.propertylandusetypeid.isin(single_use)]
    
    # Restrict df to only those properties with at least 1 bath & bed and 350 sqft area
    df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull())\
            & (df.calculatedfinishedsquarefeet>350)]

    # Handle missing values i.e. drop columns and rows based on a threshold
    df = handle_missing_values(df)
    
    # Add column for counties
    df['county'] = np.where(df.fips == 6037, 'Los_Angeles',
                           np.where(df.fips == 6059, 'Orange', 
                                   'Ventura'))    
    # drop columns not needed
    df = remove_columns(df, ['id',
       'calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 'heatingorsystemtypeid'
       ,'propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc', 
        'censustractandblock', 'propertylandusedesc'])


    # replace nulls in unitcnt with 1
    df.unitcnt.fillna(1, inplace = True)
    
    # assume that since this is Southern CA, null means 'None' for heating system
    df.heatingorsystemdesc.fillna('None', inplace = True)
    
    # replace nulls with median values for select columns
    df.lotsizesquarefeet.fillna(7313, inplace = True)
    df.buildingqualitytypeid.fillna(6.0, inplace = True)

    # Columns to look for outliers
    df = df[df.taxvaluedollarcnt < 5_000_000]
    df[df.calculatedfinishedsquarefeet < 8000]
    
    # Just to be sure we caught all nulls, drop them here
    df = df.dropna()
    
    return df

In [9]:
df = wrangle_zillow()
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,heatingorsystemdesc,county
0,10711855,2.00,3.00,8.00,"2,107.00","6,037.00","34,222,559.00","-118,617,387.00","9,158.00","60,371,132.32",...,"1,972.00","249,655.00","624,139.00","2,016.00","374,484.00","7,659.36",-0.01,2017-07-07,Central,Los_Angeles
1,10711877,2.00,4.00,8.00,"1,882.00","6,037.00","34,220,261.00","-118,616,409.00","9,035.00","60,371,132.32",...,"1,972.00","253,000.00","660,000.00","2,016.00","407,000.00","8,123.91",0.02,2017-08-29,Central,Los_Angeles
2,10711888,2.00,4.00,8.00,"1,882.00","6,037.00","34,222,491.00","-118,616,854.00","9,800.00","60,371,132.32",...,"1,972.00","257,591.00","542,923.00","2,016.00","285,332.00","6,673.24",0.08,2017-04-04,Central,Los_Angeles
3,10711910,2.00,3.00,8.00,"1,477.00","6,037.00","34,221,864.00","-118,615,739.00","11,285.00","60,371,132.32",...,"1,960.00","57,968.00","78,031.00","2,016.00","20,063.00","1,116.46",-0.04,2017-03-17,Central,Los_Angeles
4,10711923,2.00,4.00,8.00,"1,918.00","6,037.00","34,220,619.00","-118,615,253.00","11,239.00","60,371,132.32",...,"1,960.00","167,869.00","415,459.00","2,016.00","247,590.00","5,239.85",-0.01,2017-03-24,Central,Los_Angeles


In [10]:
df.shape

(69730, 25)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69730 entries, 0 to 72214
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      69730 non-null  int64  
 1   bathroomcnt                   69730 non-null  float64
 2   bedroomcnt                    69730 non-null  float64
 3   buildingqualitytypeid         69730 non-null  float64
 4   calculatedfinishedsquarefeet  69730 non-null  float64
 5   fips                          69730 non-null  float64
 6   latitude                      69730 non-null  float64
 7   longitude                     69730 non-null  float64
 8   lotsizesquarefeet             69730 non-null  float64
 9   rawcensustractandblock        69730 non-null  float64
 10  regionidcity                  69730 non-null  float64
 11  regionidcounty                69730 non-null  float64
 12  regionidzip                   69730 non-null  float64
 13  r

In [14]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

### Prep Data 

In [24]:
con_vars = ['transactiondate', 'taxamount', 'landtaxvaluedollarcnt', 'taxvaluedollarcnt', 'structuretaxvaluedollarcnt','roomcnt','lotsizesquarefeet','longitude','latitude','fips', 'calculatedfinishedsquarefeet', 'bedroomcnt','bathroomcnt']

In [44]:
def scale_my_data(train, validate, test):
    scaler = StandardScaler()
    scaler.fit(train[['roomcnt','lotsizesquarefeet','longitude','latitude','fips', 'calculatedfinishedsquarefeet', 'bedroomcnt','bathroomcnt']])
    X_train_scaled = scaler.transform(train[['roomcnt','lotsizesquarefeet','longitude','latitude','fips', 'calculatedfinishedsquarefeet', 'bedroomcnt','bathroomcnt']])
    X_validate_scaled = scaler.transform(validate[['roomcnt','lotsizesquarefeet','longitude','latitude','fips', 'calculatedfinishedsquarefeet', 'bedroomcnt','bathroomcnt']])
    X_test_scaled = scaler.transform(test[['roomcnt','lotsizesquarefeet','longitude','latitude','fips', 'calculatedfinishedsquarefeet', 'bedroomcnt','bathroomcnt']])

    train[['roomcnt_scaled','lotsizesquarefeet_scaled','longitude_scaled','latitude_scaled','fips_scaled', 'calculatedfinishedsquarefeet_scaled', 'bedroomcnt_scaled','bathroomcnt_scaled']] = X_train_scaled
    validate[['roomcnt_scaled','lotsizesquarefeet_scaled','longitude_scaled','latitude_scaled','fips_scaled', 'calculatedfinishedsquarefeet_scaled', 'bedroomcnt_scaled','bathroomcnt_scaled']] = X_validate_scaled
    test[['roomcnt_scaled','lotsizesquarefeet_scaled','longitude_scaled','latitude_scaled','fips_scaled', 'calculatedfinishedsquarefeet_scaled', 'bedroomcnt_scaled','bathroomcnt_scaled']] = X_test_scaled
    
    return train, validate, test

def prep_mall(df):
    '''
    dummy var for gender into is_male
    add 'spending_class' that cut spending score into the 4 quartiles and label the new field by q1, q2, q3, q4. 
    split on target of 'spending_score'
    scale age and annual income. 
    '''
    
    df['fips'] = pd.qcut(df.fips, q=2, labels=['fip1', 'fip2'])
    train, validate, test = explore.train_validate_test_split(df, target='logerror', seed=123)
    train, validate, test = scale_my_data(train, validate, test)
        
    return df, train, validate, test

In [45]:
df, train, validate, test = prep_mall(df)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [13]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

### Question 1

### Question 2

### Question 3

### Question 4

### Question 5

## Bonus:

Compute the mean(logerror) by zipcode and the overall mean(logerror). Write a loop that will run a t-test between the overall mean and the mean for each zip code. We want to identify the zip codes where the error is significantly higher or lower than the expected error. 